In [296]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [297]:
PATH_FOLDER = 'wikispeedia_paths-and-graph/'
LINKS_DATA = PATH_FOLDER + "links.tsv"
PATH_FINISHED_DATA = PATH_FOLDER + "paths_finished.tsv"

In [298]:
links = pd.read_csv(LINKS_DATA, sep='\t', header=None, names=["linkSource", 'linkTarget'], comment='#')
path_finished = pd.read_csv(PATH_FINISHED_DATA, sep='\t', header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'], comment='#')

In [299]:
from urllib.parse import unquote

def unquote_df(df, columns):
    '''Inputs:
            df: panda dataframe
            columns: string or string array containing column names to be url-decoded
        Return:
            panda dataframe with url-decoded column names
    '''
    for column in columns:
        N = len(df[column])
        for i in range(N):
            df.loc[i,column] = unquote(df.loc[i,column])
            
    return df

In [300]:
def check_character(headline, character):
    '''Inputs:
            headline: string
            list_words: string
        Return:
            1 if the character is in the headline
            0 otherwise
    '''
    if character in headline: #
        return 1 
    else:
         return 0

In [301]:
links = unquote_df(links, ['linkSource','linkTarget'])
path_finished = unquote_df(path_finished, ['path'])
#path_finished['back'] = path_finished['path'].apply(lambda x : check_character(x, '<'))
#path_finished = path_finished[path_finished['back']==0]


A partir de maintenant, uniquement mon code (avant: recopie pour run notebook proprement)

We build a directed graph with the wikipedia articles to perform a fictional random walk on it using the pagerank alorithm, which will be our normalization for the distance metric.

In [302]:
Wikigraph=nx.DiGraph()
e=zip(links['linkSource'], links['linkTarget'])
Wikigraph.add_edges_from(e)


In [303]:
#nx.draw(Wikigraph)
#plt.show()
print(list(Wikigraph.successors('Bede')))


['Abbot', 'Dante_Alighieri', 'Durham_Cathedral', 'England', 'Great_Britain', 'Hebrew_language', 'Julius_Caesar', 'Middle_Ages', 'Music', 'Paul_of_Tarsus', 'Season', 'Virgil']


In [304]:
GooglePageRank=nx.pagerank(Wikigraph, alpha=0.85)
print(GooglePageRank.get('United_States')) 
print(max(GooglePageRank.values())) #we see that the maximum value is reached for the US, which means it will be reached more times than any other article on a RW

0.00956554538310564
0.00956554538310564


In [305]:
def convert(string):
    return list(string.split(';'))

path_finished['path']=path_finished['path'].apply(lambda x:convert(x))

In [306]:
def count_a(article, goal, paths):
    '''This function counts the number of times an article was encountered on all paths with a specified goal
    Input: article encountered, goal of the paths, object with all paths
    Output: number of times the article was encountered summed on all paths with same given goal'''
    count=0
    for path in paths:
        if path[-1]==goal: #checks that the last element is the correct goal
            for art in path:
                if art==article:
                    count+=1
    return count
#note that we decide to count all the times an article could appear in the same path, because it would mean that it has a more significant value than if it just appeared once

In [307]:
from itertools import cycle

In [308]:
def count_aprime(aprime, article, goal, paths):
    count=0
    if aprime=='<' or article=='<': #the comeback sign is not part of the count
        return 0

    for path in paths:
        if path[-1]==goal:
            pathcycle=cycle(path)
            next_art=next(pathcycle)
            for _ in range(len(path)-1):
                art, next_art=next_art, next(pathcycle)
                if art==article:
                    if next_art==aprime:
                            count+=1    
    return count


In [309]:
path_finished['path'][1000]

['AT&T', 'United_States', 'Agriculture', 'Vegetable']

In [310]:

count=count_a('United_States','South_America',path_finished['path'])
count #seems to work

4

In [311]:
count_successor=count_aprime('Nature','Science','Rainbow',path_finished['path'])
print(count_successor)
#note that '<' is not included in the Wikigraph
#ATTENTION: what to do when aprime then <? remove them?
#seems to work otherwise too


2


In [312]:
def posterior_click_probability(aprime, article, goal, paths, alpha=0.2):
    '''Calculates the posterior click probability to reach an article given the previous article and the goal, after seeing all the data
    Input: aprime: article on which the proba is done, article:previous article, goal:final article, paths:evaluated on all those paths,
    alpha:Dirichlet parameter representing initial confidence in uniform prior distribution
    Output: posterior click probability for aprime, given article and goal'''
    
    k_a=len(list(Wikigraph.successors(article)))#number of out-degree links for article
    if count_a(article, goal, paths)!=0 and count_aprime(aprime, article, goal, paths):
        proba=(count_aprime(aprime, article, goal, paths)+alpha)/(count_a(article, goal, paths)+alpha*k_a)
    else: proba=1 #because of the log

    return proba



In [313]:
#proba=posterior_click_probability('Time','14th_century','Rainbow',path_finished['path'])
proba=posterior_click_probability('Dutch_language','Darth_Vader','Roman_Catholic_Church',path_finished['path'])
proba
#seems to work as well


0.13636363636363635

How to determine alpha parameter???

In [314]:
def path_distance(a_i, goal, path):
    sum=0
    i=-1
    if a_i!=goal:
        for a in path:
            if a==a_i:
                i=path.index(a)
        if i==-1:
            print('Error: The article looked for is not in the path')
            return 0

        pathcycle=cycle(path[i:])
        next_a=next(pathcycle)
        for _ in range(len(path)-1):
            a_i, next_a=next_a, next(pathcycle)
            if a_i!='<':
                p=posterior_click_probability(next_a, a_i, goal, path_finished['path'])
                sum-=np.log(p)
    
    return sum/(-np.log(GooglePageRank.get(goal)))
                
    

In [315]:
path_distance('15th_century','African_slave_trade',path_finished['path'][1])

Error: The article looked for is not in the path


0

In [337]:
def semantic_distance(article, goal,paths):
    dist=0
    m=0
    pat=[]

    for path in paths:
        if path[-1]==goal:
            for a in path:
                if a==article:
                    dist+=path_distance(article, goal, path)
                    m+=1
                    pat.append(path)

    if m==0: 
        print('Error, no such path exists')
        return 0

    return dist/m #, m, pat
    


In [317]:
#sdist1, m1, pat1=semantic_distance('Noam_Chomsky','Linguistics',path_finished['path'])
#sdist2, m2, pat2=semantic_distance('Renaissance','Rainbow',path_finished['path'])
#sdist5, m5, pat5=semantic_distance('Physics','Rainbow',path_finished['path'])
#sdist6, m6, pat6=semantic_distance('Light','Rainbow',path_finished['path'])
#sdist7, m7, pat7=semantic_distance('United_States', 'United_States_Constitution', path_finished['path'])
#sdist8, m8, pat8=semantic_distance('Batman','Superman',path_finished['path'])
sdist9, m9, pat9=semantic_distance('Linguistics', 'Noam_Chomsky', path_finished['path'])
sdist10, m10, pat10=semantic_distance('Language', 'Noam_Chomsky', path_finished['path'])
sdist11, m11, pat11=semantic_distance('Communication', 'Noam_Chomsky', path_finished['path'])
print(sdist9, sdist11, sdist10)


0.2367916565801112 0.30504371738085306 0.4370001363732602


In [318]:
Wikigraph.has_node('World_War_I')

True

In [319]:
for i in range(len(path_finished['path'])):
    for article in path_finished['path'][i]:
        if article=='World_War_I':
            print(path_finished['path'][i])

['Apollo', 'Superman', 'World_War_I', 'Woodrow_Wilson', 'William_Howard_Taft', 'Theodore_Roosevelt', 'William_McKinley', '<', 'William_McKinley', 'Grover_Cleveland', 'Chester_A._Arthur']
['Art', 'Albert_Einstein', 'World_War_I', 'United_States', 'United_States_Constitution']
['Art', 'Mind', 'Love', 'Human', 'Technology', 'Dolphin', 'Whale', 'Milk', 'Meat', 'Food', 'Season', '<', 'Famine', 'Democracy', 'Liberal_democracy', 'Property', 'John_Locke', 'Bristol', 'England', 'British_Empire', 'Football_(soccer)', 'FIFA', 'World_War_I', 'Sarajevo', 'Ottoman_Empire', 'Siege', 'Leonardo_da_Vinci', 'Horse', 'Art', '<', 'Human', '<', 'Clothing', 'Insect', 'Hymenoptera', 'Triassic', 'Biosphere', 'Ecology', 'Cell_(biology)', 'Ostrich', 'Bird', 'Reptile', 'Heat', 'Hippocrates', 'William_Shakespeare', 'Poetry', 'Rapping', 'Jazz', 'Atlantic_slave_trade', 'Sugar', 'Food', '<', 'Brazil', 'Africa', 'Irrigation', '<', 'Rainforest', 'Australia', 'Platypus', 'Protein', 'Potassium', 'Plant', 'Seed', 'Fruit',

In [345]:
for i in range(len(path_finished['path'])):
    if path_finished['path'][i][-1]=='Rainbow':
        print(path_finished['path'][i])
        #print(i)

['14th_century', 'Time', 'Isaac_Newton', 'Light', 'Color', 'Rainbow']
['14th_century', 'Time', 'Light', 'Rainbow']
['14th_century', '15th_century', 'Plato', 'Nature', 'Ultraviolet', 'Color', 'Rainbow']
['14th_century', 'Time', 'Science', 'Nature', 'Weather', 'Sunlight', '<', 'Sun', "Earth's_atmosphere", 'Ultraviolet', 'Color', 'Light', 'Rainbow']
['14th_century', 'Christianity', 'Bible', 'God', 'Nature', "Earth's_atmosphere", 'Ultraviolet', 'Optical_fiber', 'Light', 'Rainbow']
['14th_century', 'Time', 'Astronomy', 'Light', 'Rainbow']
['14th_century', 'Renaissance', 'Empiricism', 'Nature', 'Weather', 'Sunlight', '<', 'Rain', '<', 'Sunlight', "Earth's_atmosphere", '<', '<', 'Sun', 'Isaac_Newton', 'Rainbow']
['14th_century', 'Renaissance', 'Leonardo_da_Vinci', 'Water', 'Rain', 'Cloud', '<', '<', 'Rainbow']
['14th_century', 'Renaissance', 'Leonardo_da_Vinci', 'Water', 'Rainbow']
['14th_century', 'Europe', 'Republic_of_Ireland', '<', '<', 'Europe', '<', 'Europe', 'Republic_of_Ireland', '<',

To check our semantic distance, we will make a function that returns the top 10 most related articles to one given goal article.

In [363]:
def top_20(goal, paths):
    '''This function computes the 10 closest articles to a given goal article.
    Input: article: goal article, paths: all paths on which the distance will be measured
    Output: list of 10 closest articles to the goal or less if not enough distances have been measured'''
    top_20=[0 for _ in range(10)]
    dist=[]
    articles=[]
    already_calculated=False

    for i in range(len(paths)):
        if paths[i][-1]==goal:
            for article in paths[i]:
                already_calculated=False
                if article !='<' and article!=goal:
                    if len(articles)>0:
                        for art in articles:
                            if art==article:
                                already_calculated=True #to not calculate many times the same distance
                                
                    if already_calculated==False:
                        dist.append((semantic_distance(article, goal, paths),article))
                        print('Distance between {:} and {:} has been calculated.'.format(article, goal))
                        articles.append(article)
    
    dist.sort()
    if len(dist)>=20:
        top_20=dist[:20]
    else:
       for j in range(len(dist)):
        top_20[j]=dist[j]

    return top_20

In [357]:
top=top_20('Noam_Chomsky', path_finished['path'])
top

Distance between Canada and Noam_Chomsky has been calculated.
Distance between United_States and Noam_Chomsky has been calculated.
Distance between Education_in_the_United_States and Noam_Chomsky has been calculated.
Distance between Music and Noam_Chomsky has been calculated.
Distance between Language and Noam_Chomsky has been calculated.
Distance between Communication and Noam_Chomsky has been calculated.
Distance between Linguistics and Noam_Chomsky has been calculated.
Distance between Noam_Chomsky and Noam_Chomsky has been calculated.
Distance between Hurricane_Vince_(2005) and Noam_Chomsky has been calculated.
Distance between Spain and Noam_Chomsky has been calculated.
Distance between France and Noam_Chomsky has been calculated.
Distance between 18th_century and Noam_Chomsky has been calculated.
Distance between 20th_century and Noam_Chomsky has been calculated.
Distance between Computer and Noam_Chomsky has been calculated.
Distance between Charles_Babbage and Noam_Chomsky has

[(0.0, 'Noam_Chomsky'),
 (0.10443895274732024, 'Scheme_programming_language'),
 (0.16730024036998317, 'Psychology'),
 (0.2367916565801112, 'Linguistics'),
 (0.30504371738085306, 'Communication'),
 (0.31027981651003655, 'Tourette_syndrome'),
 (0.36232543086334995, 'Stephen_Jay_Gould'),
 (0.392790541298524, 'La_Paz'),
 (0.4370001363732602, 'Language'),
 (0.526264526967477, 'Arab-Israeli_conflict'),
 (0.526264526967477, 'The_Holocaust'),
 (0.6607988370652125, 'Alternating_current'),
 (0.6607988370652125, 'Radio_frequency'),
 (0.7256788032770468, 'Socialism'),
 (0.770277274205617, 'Sound'),
 (0.8757147883850194, 'Alphabet'),
 (0.9047884224331657, 'Philosophy'),
 (0.9127820967019243, 'Music'),
 (0.9467907452366382, 'Jew'),
 (0.9835366208061289, 'World_War_II')]

In [358]:
top_rainbow=top_20('Rainbow', path_finished['path'])
top_rainbow

Distance between 14th_century and Rainbow has been calculated.
Distance between Time and Rainbow has been calculated.
Distance between Isaac_Newton and Rainbow has been calculated.
Distance between Light and Rainbow has been calculated.
Distance between Color and Rainbow has been calculated.
Distance between Rainbow and Rainbow has been calculated.
Distance between 15th_century and Rainbow has been calculated.
Distance between Plato and Rainbow has been calculated.
Distance between Nature and Rainbow has been calculated.
Distance between Ultraviolet and Rainbow has been calculated.
Distance between Science and Rainbow has been calculated.
Distance between Weather and Rainbow has been calculated.
Distance between Sunlight and Rainbow has been calculated.
Distance between Sun and Rainbow has been calculated.
Distance between Earth's_atmosphere and Rainbow has been calculated.
Distance between Christianity and Rainbow has been calculated.
Distance between Bible and Rainbow has been calcul

[(0.0, 'Rainbow'),
 (0.13533395863614237, 'Flood'),
 (0.1644424231958515, 'Color'),
 (0.27066791727228473, 'Tornado'),
 (0.2913948188517175, 'Chromatic_aberration'),
 (0.33649855667110407, 'Light'),
 (0.3768857341069771, 'Eye'),
 (0.3939502973512229, 'Special_relativity'),
 (0.3996987849750392, 'Cloud'),
 (0.40997875023346164, 'Oxygen'),
 (0.41227450107043384, 'Microscope'),
 (0.42658870797415593, 'Crystal'),
 (0.4833404483025975, 'Wave'),
 (0.4873681016507162, 'Photon'),
 (0.49944037030146765, 'Tin'),
 (0.5005804619161088, 'Brain'),
 (0.5417140815235864, 'Optical_fiber'),
 (0.5417140815235864, 'Solar_eclipse'),
 (0.5445328510050803, 'Hurricane_Georges'),
 (0.5796389476142207, 'Isaac_Newton')]

In [359]:
top_astro=top_20('Astronomy', path_finished['path'])
top_astro

Distance between Hydrogen and Astronomy has been calculated.
Distance between Star and Astronomy has been calculated.
Distance between Astrology and Astronomy has been calculated.
Distance between Astronomy and Astronomy has been calculated.
Distance between Saturn_V and Astronomy has been calculated.
Distance between Asteroid and Astronomy has been calculated.
Distance between Telescope and Astronomy has been calculated.
Distance between Whale and Astronomy has been calculated.
Distance between Technology and Astronomy has been calculated.
Distance between Space_exploration and Astronomy has been calculated.
Distance between Universe and Astronomy has been calculated.
Distance between Galaxy and Astronomy has been calculated.
Distance between Currency and Astronomy has been calculated.
Distance between 18th_century and Astronomy has been calculated.
Distance between Time and Astronomy has been calculated.
Distance between Science and Astronomy has been calculated.
Distance between Rom

[(0.0, 'Astronomy'),
 (0.20046760576919972, 'Telescope'),
 (0.21926671228701783, 'Nostradamus'),
 (0.27239423496618353, 'Asteroid'),
 (0.3853736523815628, 'Extraterrestrial_life'),
 (0.4973050004476167, 'Electric_field'),
 (0.49792144043459646, 'Galileo_Galilei'),
 (0.5008078761808197, 'Space_Invaders'),
 (0.5087129320753612, 'Saturn_V'),
 (0.5100214257537568, 'Space_exploration'),
 (0.5169199242620135, 'Stephen_Hawking'),
 (0.5302140797019032, 'Aristotle'),
 (0.5722164614047334, "Hubble's_law"),
 (0.5777212170446819, 'Nicolaus_Copernicus'),
 (0.6151356152177155, 'Astrology'),
 (0.6155452285459225, 'Poland'),
 (0.6770777638947565, 'Archaeology'),
 (0.6810149554552594, 'Roman_Empire'),
 (0.7195427540839443, 'England'),
 (0.7402728176380172, 'Hydrogen')]

In [361]:
top_20('Physics',path_finished['path'])

Distance between Citrus and Physics has been calculated.
Distance between Vitamin_C and Physics has been calculated.
Distance between Linus_Pauling and Physics has been calculated.
Distance between Quantum_chemistry and Physics has been calculated.
Distance between Physics and Physics has been calculated.
Distance between United_States and Physics has been calculated.
Distance between Computer and Physics has been calculated.
Distance between Automobile and Physics has been calculated.
Distance between Energy and Physics has been calculated.
Distance between Vitamin and Physics has been calculated.
Distance between Folic_acid and Physics has been calculated.
Distance between Cancer and Physics has been calculated.
Distance between DNA and Physics has been calculated.
Distance between Cell_(biology) and Physics has been calculated.
Distance between Microscope and Physics has been calculated.
Distance between Science and Physics has been calculated.
Distance between Fruit and Physics has

[(0.0, 'Physics'),
 (0.20052723759962435, 'Force'),
 (0.22354459534301582, 'Gravitation'),
 (0.2568894606330325, 'Motion_(physics)'),
 (0.277807183162991, 'Quantum_chemistry'),
 (0.2901741675711402, 'Crash_test_dummy'),
 (0.3802348466338317, 'Engineering'),
 (0.3938470259930577, 'Technology'),
 (0.4010447603598517, 'Dam'),
 (0.4326827294163586, 'Neutron'),
 (0.4392102621411392, 'George_III_of_the_United_Kingdom'),
 (0.488457362529619, 'Science'),
 (0.5366285481765735, 'Quantum_mechanics'),
 (0.5399483519413473, 'Lance_Armstrong'),
 (0.5859845896870155, 'Essential_oil'),
 (0.616837441156671, 'Polonium'),
 (0.6237502985335727, 'Asteroid'),
 (0.6294456987549262, 'Solar_System'),
 (0.6684396475664381, 'University'),
 (0.6888564674561402, 'Semiconductor')]

In [362]:
top_20('Biology', path_finished['path'])

Distance between Biotechnology and Biology has been calculated.
Distance between Biology and Biology has been calculated.
Distance between Love and Biology has been calculated.
Distance between Paleontology and Biology has been calculated.
Distance between Psychology and Biology has been calculated.
Distance between Health and Biology has been calculated.
Distance between Organism and Biology has been calculated.
Distance between Nelson_Mandela and Biology has been calculated.
Distance between George_W._Bush and Biology has been calculated.
Distance between Creationism and Biology has been calculated.
Distance between Evolution and Biology has been calculated.
Distance between South_Africa and Biology has been calculated.
Distance between Biodiversity and Biology has been calculated.
Distance between Maldives and Biology has been calculated.
Distance between United_Kingdom and Biology has been calculated.
Distance between Francis_Crick and Biology has been calculated.
Distance between 

[(0.0, 'Biology'),
 (0.15105190494798362, 'Binomial_nomenclature'),
 (0.19082504834708938, 'Louis_Pasteur'),
 (0.20698818191774138, 'Love'),
 (0.22342531294641793, 'Paleontology'),
 (0.24279263368313256, 'Biotechnology'),
 (0.2557899325590062, 'Psychology'),
 (0.26771770170772985, 'Sociology'),
 (0.27764206706500477, 'Organism'),
 (0.27955733150575457, 'Bioinformatics'),
 (0.29103678702469593, 'The_Origin_of_Species'),
 (0.307521872908309, 'Glass'),
 (0.38165009669417876, 'James_D._Watson'),
 (0.3833081314842784, 'Economics'),
 (0.4021347639622707, 'Environmental_science'),
 (0.4145028234241512, 'Tropics'),
 (0.4187696066557134, 'Velociraptor'),
 (0.42635773831973345, 'Science'),
 (0.43921652965386676, 'Biodiversity'),
 (0.45407461281335204, 'Health')]